In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')

df=pd.read_csv('/content/drive/My Drive/_Universität_HPI/Semester 5/Machine Learning/Kaggle/files/train_A_derma.csv',header='infer')


Mounted at /content/drive/


In [2]:
derma_df = df
derma_df = derma_df.drop('Id', axis=1)
derma_df = derma_df.drop('Doughnuts consumption', axis=1)

In [3]:
derma_df['Lesion Area'] = 0.0
derma_df['Lesion Size'] = 0.0

In [4]:
def fillAreaAndSize(idf):
  for i in range(len(idf['Lesion Area'])):
    if (idf.at[i, 'Small'] == 1.0 or (idf.at[i, 'Mid'] == 0 and idf.at[i, 'Large'] == 0)):
      idf.at[i,'Lesion Area'] = 0
    elif (idf.at[i, 'Mid'] == 1.0 or (idf.at[i, 'Small'] == 0 and idf.at[i, 'Large'] == 0)):
      idf.at[i,'Lesion Area'] = 0.5
    elif (idf.at[i, 'Large'] == 1.0 or (idf.at[i, 'Small'] == 0 and idf.at[i, 'Mid'] == 0)):
      idf.at[i,'Lesion Area'] = 1
    else:
      idf.at[i,'Lesion Area'] = np.nan
  for i in range(len(idf['Lesion Size'])):
    if idf.at[i, 'Small size'] == 1 or (idf.at[i, 'Mid size'] == 0 and idf.at[i, 'Large size'] == 0):
      idf.at[i,'Lesion Size'] = 0
    elif idf.at[i, 'Mid size'] == 1 or (idf.at[i, 'Small size'] == 0 and idf.at[i, 'Large size'] == 0):
      idf.at[i,'Lesion Size'] = 0.5
    elif idf.at[i, 'Large size'] == 1 or (idf.at[i, 'Small size'] == 0 and idf.at[i, 'Mid size'] == 0):
      idf.at[i,'Lesion Size'] = 1
    else:
      idf.at[i,'Lesion Size'] = np.nan
  for i in range(len(idf['Lesion Area'])):
    if (np.isnan(idf.at[i, 'Lesion Area']) and not np.isnan(idf.at[i, 'Lesion Size'])):
      idf.at[i, 'Lesion Area'] = idf.at[i, 'Lesion Size']
    elif (np.isnan(idf.at[i, 'Lesion Size']) and not np.isnan(idf.at[i, 'Lesion Area'])):
      idf.at[i, 'Lesion Size'] = idf.at[i, 'Lesion Area']
    elif (np.isnan(idf.at[i, 'Lesion Area']) and np.isnan(idf.at[i, 'Lesion Size'])):
      idf.at[i, 'Lesion Size'] = 0.5
      idf.at[i, 'Lesion Area'] = 0.5

fillAreaAndSize(derma_df)
derma_df = derma_df.drop(['Small','Mid','Large','Small size', 'Mid size', 'Large size'], axis=1)

In [5]:
for i in derma_df.columns:
  print(i, derma_df[i].isna().sum())

Lession 190
Genetic Propensity 194
Skin X test 175
Skin color 177
Fake/Real 0
Lesion Area 0
Lesion Size 0


In [6]:
base_df = derma_df[['Lesion Area','Lesion Size','Fake/Real', 'Genetic Propensity']]#, 'Doughnuts consumption']]
base_df['Fake/Real'] = base_df['Fake/Real'].replace({'real': 0, 'fake': 1})
base_df

<ipython-input-6-f316d78b47d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df['Fake/Real'] = base_df['Fake/Real'].replace({'real': 0, 'fake': 1})


,Lesion Area,Lesion Size,Fake/Real,Genetic Propensity
0,0.5,0.5,0,-1.27713
1,0.0,0.0,0,-1.84207
2,0.5,0.5,0,-1.82290
3,0.0,0.0,0,-1.36049
4,1.0,1.0,1,NaN
...,...,...,...,...
592,0.0,0.5,1,-2.16422
593,0.0,0.0,0,NaN
594,1.0,1.0,0,-1.03834
595,0.0,0.0,0,NaN


In [7]:
#Genetic Propensity auffüllen
from sklearn.ensemble import RandomForestRegressor


def imputeCategories(target_variable, base):
  # Split the dataset into two parts: one with missing values and one without
  data_missing = base[base[target_variable].isnull()]
  data_not_missing = base[base[target_variable].notnull()]

  # Separate the target variable and the predictors for the two datasets
  X_missing = data_missing.drop(target_variable, axis=1).values
  y_missing = data_missing[target_variable].values
  X_not_missing = data_not_missing.drop(target_variable, axis=1).values
  y_not_missing = data_not_missing[target_variable].values

  # Create a RandomForestRegressor model for imputation
  imputation_model = RandomForestRegressor(n_estimators=100, random_state=42)

  # Train the imputation model on the data without missing values
  imputation_model.fit(X_not_missing, y_not_missing)

  # Use the trained model to impute missing values
  imputed_values = imputation_model.predict(X_missing)

  # Update the original dataset with the imputed values
  base.loc[base[target_variable].isnull(), target_variable] = imputed_values

imputeCategories('Genetic Propensity',base_df)
base_df['Lession'] = derma_df['Lession']
imputeCategories('Lession',base_df)
base_df['Lession'] = base_df['Lession'].round(0)
base_df['Skin X test'] = derma_df['Skin X test']
imputeCategories('Skin X test',base_df)
base_df['Skin X test'] = base_df['Skin X test'].round(0)
base_df['Skin color'] = derma_df['Skin color']
imputeCategories('Skin color',base_df)
base_df['Skin color'] = base_df['Skin color'].round(0)

<ipython-input-7-509567f59109>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df['Lession'] = derma_df['Lession']
<ipython-input-7-509567f59109>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df['Lession'] = base_df['Lession'].round(0)
<ipython-input-7-509567f59109>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [8]:
base_df

,Lesion Area,Lesion Size,Fake/Real,Genetic Propensity,Lession,Skin X test,Skin color
0,0.5,0.5,0,-1.277130,1.0,4.0,0.0
1,0.0,0.0,0,-1.842070,0.0,3.0,0.0
2,0.5,0.5,0,-1.822900,0.0,3.0,0.0
3,0.0,0.0,0,-1.360490,1.0,3.0,0.0
4,1.0,1.0,1,-1.761755,0.0,2.0,0.0
...,...,...,...,...,...,...,...
592,0.0,0.5,1,-2.164220,0.0,2.0,1.0
593,0.0,0.0,0,-1.684823,0.0,3.0,1.0
594,1.0,1.0,0,-1.038340,0.0,3.0,0.0
595,0.0,0.0,0,-1.684823,0.0,3.0,1.0


In [9]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

def kFoldCrossVal(model, k, name):
  # Create a KFold cross-validation object
  kf = KFold(n_splits=k, shuffle=True, random_state=42)
  # Initialize a list to store the evaluation results
  results = []

  # Perform k-fold cross-validation
  for train_index, test_index in kf.split(X):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]
      # Fit the model on the training data
      model.fit(X_train, y_train)
      # Make predictions on the test data
      y_pred = model.predict(X_test)
      # Evaluate the model's performance using an appropriate metric (e.g., accuracy)
      accuracy = accuracy_score(y_test, y_pred)
      results.append(accuracy)
  # Calculate the mean and standard deviation of the results
  mean_accuracy = sum(results) / len(results)
  std_deviation = np.std(results)
  print(name)
  print(f"Mean Accuracy: {mean_accuracy}")
  print(f"Standard Deviation: {std_deviation}")

In [10]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

X = base_df.drop(['Fake/Real','Lesion Area', 'Lesion Size', 'Lession'],axis=1).values
y = base_df['Fake/Real'].values

#rfc = RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=1000)
#kFoldCrossVal(rfc, 20, 'RandomForestClassifier')

dtc = DecisionTreeClassifier(random_state=42,ccp_alpha=0.01, class_weight="balanced",criterion="entropy")
kFoldCrossVal(dtc, 20, 'DecisionTreeClassifier')

#dtc = DecisionTreeClassifier(ccp_alpha=0.01, class_weight="balanced",criterion="entropy")
#kFoldCrossVal(dtc, 20, 'DecisionTreeClassifier')


#xgbm = xgb.XGBClassifier()
#kFoldCrossVal(xgbm, 20, 'XGBClassifier')

DecisionTreeClassifier
Mean Accuracy: 0.8959770114942532
Standard Deviation: 0.05323651040806545


In [11]:
df_test = pd.read_csv('/content/drive/My Drive/_Universität_HPI/Semester 5/Machine Learning/Kaggle/files/test_A_derma.csv',header='infer')
df_test['Lesion Area'] = 0.0
df_test['Lesion Size'] = 0.0
fillAreaAndSize(df_test)
df_test = df_test.drop(['Small','Mid','Large','Small size', 'Mid size', 'Large size'], axis=1)
base_test = df_test[['Lesion Area','Lesion Size','Fake/Real', 'Genetic Propensity', 'Doughnuts consumption']]
base_test['Fake/Real'] = base_test['Fake/Real'].replace({'real': 0, 'fake': 1})

KeyError: ignored

In [ ]:
pip install -U imbalanced-learn

In [ ]:
# Import necessary libraries
import spacy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Load spaCy NLP model
nlp = spacy.load('en_core_web_sm')

# Load your dataset containing headlines and true/fake labels
data = pd.read_csv('/content/drive/My Drive/_Universität_HPI/Semester 5/Machine Learning/Kaggle/files/train_B_text.csv')

# Preprocess the text data
#def preprocess_text(text):
#    doc = nlp(text)
#    processed_text = " ".join([token.lemma_ for token in doc if not token.is_stop])
#    return processed_text

#data['Title'] = data['Title'].apply(preprocess_text)
def starts_with_quotation(text):
    return int(text.startswith('"'))

# Create a new feature 'StartsWithQuotation' based on the 'Title' feature
data['StartsWithQuotation'] = data['Title'].apply(starts_with_quotation)


# Split the data into training and testing sets
X = data[['Title', 'StartsWithQuotation']]
y = data['Fake/Real'].map({'fake': 0, 'real': 1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


# Create an oversampler
#oversampler = RandomOverSampler(random_state=43)
#smote = SMOTE(random_state=42)

# Resample the data
#X_train_reshaped = X_train.values.reshape(-1, 1)
#X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_reshaped, y_train)

#pipeline = Pipeline([
#    ('tfidf', TfidfVectorizer(max_features=5000)),
#    ('smote', SMOTE(random_state=42))
#])

# Apply the pipeline to your text data
#X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_reshaped, y_train)


#X_train_resampled = X_train_resampled.ravel()

# Vectorize the text data using TF-IDF
#tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the max_features
#X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_resampled)
#X_test_tfidf = tfidf_vectorizer.transform(X_test)

#############


tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)



# Train a logistic regression classifier
clf = RandomForestClassifier()
#clf.fit(X_train_tfidf, y_train_resampled)
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf.values)
accuracy = accuracy_score(y_test, y_pred)
print(f"1. Accuracy: {accuracy}")





#confusion = confusion_matrix(y_test, y_pred)
#classification_rep = classification_report(y_test, y_pred)


#print("Confusion Matrix:")
#print(confusion)
#print("Classification Report:")
#print(classification_rep)

In [ ]:
data = pd.read_csv('/content/drive/My Drive/_Universität_HPI/Semester 5/Machine Learning/Kaggle/files/train_B_text.csv')

def starts_with_quotation(text):
    return int(text.startswith('"'))

# Create a new feature 'StartsWithQuotation' based on the 'Title' feature
data['StartsWithQuotation'] = data['Title'].apply(starts_with_quotation)


# Split the data into training and testing sets
X = data[['Title', 'StartsWithQuotation']]
y = data['Fake/Real'].map({'fake': 0, 'real': 1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X

In [ ]:
import spacy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load spaCy NLP model
nlp = spacy.load('en_core_web_sm')

# Load your dataset containing headlines and true/fake labels
data = pd.read_csv('/content/drive/My Drive/_Universität_HPI/Semester 5/Machine Learning/Kaggle/files/train_B_text.csv')

# Preprocess the text data
def preprocess_text(text):
    doc = nlp(text)
    processed_text = " ".join([token.lemma_ for token in doc if not token.is_stop])
    return processed_text

data['Title'] = data['Title'].apply(preprocess_text)


# Extract additional features
data['text_length'] = data['Title'].apply(len)
# You can add more features like word count, punctuation count, sentiment scores, etc.

# Split the data into training and testing sets
X = data[['Title', 'text_length']]  # Features including text and text length
y = data['Fake/Real']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a ColumnTransformer to preprocess text and scale numerical features
text_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(max_features=5000))  # You can adjust the max_features
])

num_features = ['text_length']  # Add other numerical features as needed

preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, 'Title'),
        ('num', StandardScaler(), num_features)
    ])

# Combine the preprocessing with a classifier in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(class_weight='balanced'))])

# Train the model
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(confusion)
print("Classification Report:")
print(classification_rep)